In [3]:
import os, shutil, sys
# os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

import numpy as np
import numpy.random as rand
import matplotlib.pyplot as mp
import keras
import keras.backend as K
import keras.preprocessing as kpre
import keras.datasets as kdat
import keras.models as kmod
import keras.layers as klay
import keras.optimizers as kopt
import keras.utils as kutil
import keras.callbacks as kcall
import keras.initializers as kinit
import keras.regularizers as kreg
import keras.applications as kapp
import sklearn.model_selection as slms
import tensorflow as tf
import pandas as pd
import multiprocessing
from numba import jit
from skimage.color import rgb2gray
from sklearn.decomposition import PCA
from tqdm import tnrange, tqdm_notebook as tqdm
from keras_tqdm import TQDMNotebookCallback as ktqdm
from jupyterthemes import jtplot

In [4]:
# tf.enable_eager_execution()
jtplot.style()
mp.style.use("seaborn")
mp.rcParams["axes.axisbelow"] = True
mp.rcParams["text.usetex"] = True
mp.rcParams["font.family"] = "serif"
mp.rcParams["figure.figsize"] = (16/2, 9/2)
mp.rcParams["figure.dpi"] = 100
mp.rcParams["text.latex.preamble"] = [r"\usepackage{physics}"]

In [5]:
np.random.seed(1000)

In [6]:
width = 32
height = 32
batch_size = 10
nb_epochs = 15
code_length = 128

In [8]:
(X_train, Y_train), (X_test, Y_test) = kdat.cifar10.load_data()

In [9]:
graph = tf.Graph()

with graph.as_default():
    # Global step
    global_step = tf.Variable(0, trainable=False)
    
    # Input batch
    input_images = tf.placeholder(tf.float32, shape=(batch_size, height, width, 3))

    # Convolutional layer 1
    conv1 = tf.layers.conv2d(inputs=input_images,
                             filters=32,
                             kernel_size=(3, 3),
                             kernel_initializer=tf.contrib.layers.xavier_initializer(),
                             activation=tf.nn.tanh)

    # Convolutional output (flattened)
    conv_output = tf.contrib.layers.flatten(conv1)

    # Code layer
    code_layer = tf.layers.dense(inputs=conv_output,
                                 units=code_length,
                                 activation=tf.nn.tanh)
    
    # Code output layer
    code_output = tf.layers.dense(inputs=code_layer,
                                  units=(height - 2) * (width - 2) * 3,
                                  activation=tf.nn.tanh)

    # Deconvolution input
    deconv_input = tf.reshape(code_output, (batch_size, height - 2, width - 2, 3))

    # Deconvolution layer 1
    deconv1 = tf.layers.conv2d_transpose(inputs=deconv_input,
                                         filters=3,
                                         kernel_size=(3, 3),
                                         kernel_initializer=tf.contrib.layers.xavier_initializer(),
                                         activation=tf.sigmoid)
    
    # Output batch
    output_images = tf.cast(tf.reshape(deconv1, 
                                       (batch_size, height, width, 3)) * 255.0, tf.uint8)

    # Reconstruction L2 loss
    loss = tf.nn.l2_loss(input_images - deconv1)

    # Training operations
    learning_rate = tf.train.exponential_decay(learning_rate=0.0005, 
                                               global_step=global_step, 
                                               decay_steps=int(X_train.shape[0] / (2 * batch_size)), 
                                               decay_rate=0.95, 
                                               staircase=True)
    
    trainer = tf.train.RMSPropOptimizer(learning_rate)
    training_step = trainer.minimize(loss)

W0822 22:55:09.179143  4732 module_wrapper.py:136] From D:\ProgramData\Anaconda3\envs\compsense\lib\site-packages\tensorflow_core\python\util\module_wrapper.py:163: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0822 22:55:09.182143  4732 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0822 22:55:09.184143  4732 deprecation.py:323] From <ipython-input-9-ecee374bae62>:15: conv2d (from tensorflow.python.layers.convolutional) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
W0822 22:55:09.187144  4732 deprecation.py:323] From D:\ProgramData\Anaconda3

In [11]:
use_gpu = True

config = tf.ConfigProto(intra_op_parallelism_threads=multiprocessing.cpu_count(), 
                        inter_op_parallelism_threads=multiprocessing.cpu_count(), 
                        allow_soft_placement=True, 
                        device_count = {'CPU' : 1, 
                                        'GPU' : 1 if use_gpu else 0})

session = tf.InteractiveSession(graph=graph, config=config)

D:\ProgramData\Anaconda3\envs\compsense\lib\site-packages\tensorflow_core\python\client\session.py:1741: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [12]:
tf.global_variables_initializer().run()

In [13]:
@jit
def create_batch(t, gray=False):
    X = np.zeros((batch_size, height, width, 3 if not gray else 1), dtype=np.float32)
        
    for k, image in enumerate(X_train[t:t+batch_size]):
        if gray:
            X[k, :, :, :] = rgb2gray(image)
        else:
            X[k, :, :, :] = image / 255.0
        
    return X

In [14]:
for e in range(nb_epochs):
    total_loss = 0.0
    
    for t in range(0, X_train.shape[0], batch_size):
        feed_dict = {
            input_images: create_batch(t)
        }

        _, v_loss = session.run([training_step, loss], feed_dict=feed_dict)
        total_loss += v_loss
        
    print('Epoch {} - Total loss: {}'.format(e+1, total_loss))

Epoch 1 - Total loss: 967222.9230499268
Epoch 2 - Total loss: 641915.259223938
Epoch 3 - Total loss: 624764.7482757568


KeyboardInterrupt: 